# CityBikes

Send a request to CityBikes for the city of your choice. 

In [34]:
import pandas as pd
import requests
# 
#url = "http://api.citybik.es/v2/networks/velobike-moscow" /API endpoint URL (velobike-moscow) - didn't work for me
url="http://api.citybik.es/v2/networks/bixi-toronto" # /API endpoint URL (bixi-toronto)
response = requests.get(url)
if response.status_code == 200: # Check if the request was successful
    data_dict = response.json() # Parse JSON data into a Python dictionary
    stations = data_dict["network"]["stations"] # Extract the list of stations from the dictionary
    df = pd.DataFrame(stations) # Create a DataFrame
else:
    print("Failed to fetch data from the API.")


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [35]:
bike_stations=[] # creating empty bike_stations list where will collect requested info
for station in stations: # starting for loop to go through all rows in stations by filtering requested fields 
    station_id=station['id']
    name=station['name']
    latitude=station['latitude']
    longitude=station['longitude']
    number_of_bikes=station['free_bikes']
    # append returned values to bike_station list. 
    bike_stations.append({"Station_id":station_id,
                          "Name":name,
                          "Latitude":latitude,
                          "Longitude":longitude,
                          "Free Bikes": number_of_bikes
                          
                         })

Put your parsed results into a DataFrame.

In [36]:
# create a dafaframe from bike_stations list
df_bike=pd.DataFrame(bike_stations)
# Store the DataFrame in the IPython store
%store df_bike
#df.to_csv('city_bikes_ll_2.csv',index=False) // was using/testing "csv" file as an alterantive source for data

Stored 'df_bike' (DataFrame)


In [39]:
df_bike # testing dataframe

,Station_id,Name,Latitude,Longitude,Free Bikes
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,16
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693,20
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,7
...,...,...,...,...,...
669,7f760f7680dae3ad47ef0821337c8366,Scarborough Soccer Centre,43.715466,-79.287660,0
670,6f7157a5846ff850e7e5b9f6b8b450c4,Lansdowne Ave / Seaforth Ave,43.642904,-79.437461,3
671,cc5bd0485cb1b99156098277af99641f,Danforth Rd / Birchmount Rd,43.706880,-79.268510,0
672,362434f77c0efeb50a5deb4ca9ce8969,Pharmacy Ave / Teesdale Pl,43.697356,-79.285287,2
